In [3]:
import os
import math
import re
import sys
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [27]:
class OptionsData:
    s_data_dir = "data"
    s_default_filename = "isx2010C.xls"
    
    def __init__(self, filename=s_default_filename, clean=True):
        filepath = os.path.join(self.s_data_dir, filename)
        if not os.path.isfile(filepath):
            faulty_filepath = filepath
            filepath = os.path.join(self.s_data_dir, self.s_default_filename)
            print(f"Warning: could not find {faulty_filepath!r}; proceeding with {filepath!r}")
        self.__m_sheet_df_dict = pd.read_excel(filepath, sheet_name=None)
        if clean:
            for key, val in self.__m_sheet_df_dict.items():
                self.__m_sheet_df_dict[key] = self.__clean_df(val)
                
    def __get_item__(self, key):
        return self.__m_sheet_df_dict[key]
    
    def get_sheet_df_dict(self):
        return self.__m_sheet_df_dict
    
    def get_df_with_strikes(self, E, sheet_name=""):
        if not sheet_name:
            sheet_name = list(self.__m_sheet_df_dict.keys())[0]
            print(f"Warning: sheet name not specified; proceeding with {sheet_name!r}")
        df = self.__m_sheet_df_dict[sheet_name]
        common = ["T", "T_norm", "S", "r"]
        strikes_search_pattern = "|".join(map(lambda x: f"({str(x)})", [E] if type(E) is not list else E))
        cols = [*common, *filter(lambda x: re.search(strikes_search_pattern, str(x)), df.columns)]
        return df[cols]
    
    def __clean_df(self, df):
        # Discard rows where no options data is available.
        df = df.dropna(how="all")
        # Rename the columns according to the following convention:
        #  T = Time to Maturity
        #  S = Price of the Underlying
        #  r = Risk-free interest rate
        df = df.rename(lambda x: self.__rename_df_cols(str(x), df), axis="columns")
        # Adjust the interest rate properly.
        df["r"] = df["r"] / 100
        # Add new column with annual-normalized T (252 = no. trading days in a year).
        df["T_norm"] = df["T"] / 252
        # Re-arrange the columns.
        common = ["T", "T_norm", "S", "r"]
        cols = [*common, *filter(lambda x: re.search("[0-9]+", str(x)), df.columns)]
        return df[cols]
    
    def __rename_df_cols(self, col_name, df):
        ncol = len(df.columns)
        # Time to maturity | (price of the underlying | risk-free rate).
        regex = r"(?P<T>[0-9]+(-[0-9]{2}){2} ([0-9]{2}:){2}[0-9]{2})|(?P<Sr>Unnamed: (?P<idx>[0-9]+))"
        match = re.match(regex, col_name)
        if not match:
            return col_name
        group_dict = match.groupdict()
        if group_dict["T"]:
            return "T"
        elif group_dict["Sr"]:
            col_idx = int(group_dict["idx"])
            # Third last depicts the price of the underlying...
            if col_idx == ncol - 3:
                return "S"
            # ...and the second last the risk free rate.
            elif col_idx == ncol - 2:
                return "r"

In [32]:
data = OptionsData()
data.get_df_with_strikes(340).tail()

,T,T_norm,S,r,340
81,5,0.019841,524.29,0.0006,185.50
82,4,0.015873,527.93,0.0006,188.55
83,3,0.011905,529.59,0.0006,190.10
84,2,0.007937,524.11,0.0005,184.10
85,1,0.003968,NaN,NaN,NaN


In [21]:
def implied_volatility(sigma0, S, E, r, T, C_obs):
    eps = np.finfo(float).eps
    d1 = (math.log(S/E) + (r + 0.5*sigma0**2) * T) / (math.sqrt(sigma0**2 * T) + eps)
    d2 = d1 - math.sqrt(sigma0**2 * T)
    C = norm.cdf(d1) * S - E * math.exp(-r * T) * norm.cdf(d2)
    return 0.5*(C - C_obs)**2

In [34]:
E = '340'
df_single = data.get_df_with_strikes(int(E)).dropna()

rows_iter = df_single.iterrows()
index, row = next(rows_iter)
S = row['S']
T = row['T_norm']
r = row['r']
sigma, *_ = minimize(implied_volatility, 1.0, args=(S, float(E), r, T, row[E]))['x']
d1 = (math.log(S/float(E)) + (r + 0.5*sigma**2)*T) / (sigma * math.sqrt(T) + np.finfo(float).eps)
delta = norm.cdf(d1)
long_prev = row[E]
short_prev = delta * S

mse = 0.0

for index, row in rows_iter:
    long = row[E]
    short = delta * S
    dlong = long - long_prev
    dshort = short - short_prev
    mse += (dlong - dshort)**2
    long_prev = long
    short_prev = short
    
    if index % 2 == 0:
        S = row['S']
        T = row['T_norm']
        r = row['r']
        
        sigma, *_ = minimize(implied_volatility, 1.0, args=(S, float(E), r, T, row[E]))['x']
        d1 = (math.log(S/float(E)) + (r + 0.5*sigma**2)*T) / (sigma * math.sqrt(T) + np.finfo(float).eps)
        delta = norm.cdf(d1)
    
mse /= df_single.shape[0]

In [35]:
mse

14336.217103243194